# Packer's Problem - from the F# Discord Server

## Description

You entrusted Petya, who works as a packer, with the responsible task of stacking boxes of different sizes
one into another, and the instructions were the following:
* The boxes must fit into each other (the total size of the boxes lying next to each other does not
exceed the size of the box in which they are put) — otherwise, they will get damaged.
* To put box B into box A, you must first open A and then open B.
* To close the box, you must first close all the boxes that are in it.
* If there are several boxes next to each other, then you need to close them in the order in which they
were opened.

Petya has already ruined a lot of boxes, but you decided to give him one last chance.

Check if Petya has packed the boxes correctly.

The nesting depth d(x) of box x that is not inside another box is 1. The nesting depth d(x) of box x that
is inside box y is d(y) + 1.

Check if Petya has packed the boxes correctly and find the maximum nesting depth among all the boxes.

### Input

The first line contains a single integer n (1 ≤ n ≤ 10 000) — the number of boxes.

Each of the following 2n lines contains a character: ‘O’ (open, i.e., Petya opens the box) or ‘C’ (close, i.e.,
Petya closes the box), and an integer s (1 ≤ s ≤ 100 000), which denotes the size of the box.

### Output

Print “Well done!” and an integer denoting the maximum nesting depth of the boxes if Petya has packed
the boxes correctly, or “You are fired!” otherwise.

## Setup

In [1]:
let inputFilePath = "input-1.txt"
// let inputFilePath = "input-fail-1.txt"

## Read in the Command Lines

In [2]:
type Command = | Open of int | Close of int | Unknown of string

let lineToCommand (line: string): Command =
    match line[0] with
    | 'O' -> Open (Int32.Parse(line[1..]))
    | 'C' -> Close (Int32.Parse(line[1..]))
    | other -> Unknown line

In [3]:
open System.IO

let lines = File.ReadLines(inputFilePath)
let commands = lines |> Seq.map lineToCommand

In [4]:
commands

[ Open 20, Open 10, Open 5, Close 5, Open 5, Open 3, Close 3, Open 2, Close 2, Close 5, Close 10, Close 20 ] f Microsoft.FSharp.Collections.SeqModule+revamp@728[System.String,FSI_0006+Command] f Microsoft.FSharp.Collections.SeqModule+Map@748-1[System.String,FSI_0006+Command] mapping FSI_0007+commands@4 ie [ O 20, O 10, O 5, C 5, O 5, O 3, C 3, O 2, C 2, C 5, C 10, C 20 ] Current <null> (values) [ O 20, O 10, O 5, C 5, O 5, O 3, C 3, O 2, C 2, C 5, C 10, C 20 ] (values) index type value 0 FSI_0006+Command+Open Open 20 Item 20 1 FSI_0006+Command+Open Open 10 Item 10 2 FSI_0006+Command+Open Open 5 Item 5 3 FSI_0006+Command+Close Close 5 Item 5 4 FSI_0006+Command+Open Open 5 Item 5 5 FSI_0006+Command+Open Open 3 Item 3 6 FSI_0006+Command+Close Close 3 Item 3 7 FSI_0006+Command+Open Open 2 Item 2 8 FSI_0006+Command+Close Close 2 Item 2 9 FSI_0006+Command+Close Close 5 Item 5 10 FSI_0006+Command+Close Close 10 Item 10 11 FSI_0006+Command+Close Close 20 Item 20

## Model the Boxes

In [5]:
type BoxState = | Opened | Closed

type Box = {state: BoxState; size: int; openedChildren: Box list; closedChildren: Box list }

let totalBoxSize (boxList: Box list): int =
    boxList |> List.map (fun box -> box.size) |> List.sum

let isCloseable (box: Box): bool =
    box.openedChildren.IsEmpty

let sizeRemaining (box: Box): int =
    box.size - (totalBoxSize box.openedChildren) - (totalBoxSize box.closedChildren)

let checkChildrenFit (box: Box): bool =
    (sizeRemaining box) >= 0

let closeFirstOpenChild (box:Box): Box =
    if (box.state = Closed)
    then box
    else
        match box.openedChildren with
        | head::tail ->
            let newState = if (tail.IsEmpty) then Closed else Opened
            {box with state = newState; openedChildren = tail; closedChildren = {head with state = Closed}::box.closedChildren}
        | other -> box

let boxSummary (box:Box) = ((if (box.state = Opened) then 'O' else 'C'), box.size, sizeRemaining box)

## Model the Command Processing

In [6]:
type BoxStack = Box list

type ProcessStatus = {stack: BoxStack; maxDepth: int}

let initProcessStatus = {stack = []; maxDepth = 0}

// Close the box that is the head of the stack.  That means:
// * If the box that is the head is closed, return the stack unchanged.
// * Remove that box that is the head of the stack (no need to a actually close it).
// * If the box that is the new head has open children,
//   close the first open child (which is same asthe old head)
//   and make the second open child (if any) the "new new" head.
let closeStackHead (stack: BoxStack): BoxStack =
    match stack with
    | [] -> [] // nothing to close
    | head::tail ->
        if (head.state = Closed)
        then stack
        else // tail is the new stack, unless it is empty
            match tail with
            | [] ->
                if (head.state = Closed)
                then stack
                else [{head with state = Closed}] // always leave the last box on the stack, so we can see if it was closed
            | head2::tail2 -> // close first open child of new top of new stack
                match head2.openedChildren with
                | [] -> tail
                | openHead::openTail ->
                    match openTail with
                    | [] ->
                        {head2 with openedChildren = []; closedChildren = {openHead with state = Closed}::head2.closedChildren}::tail2
                    | openHead2::openTail2 ->
                        openHead2::{head2 with openedChildren = openTail; closedChildren = {openHead with state = Closed}::head2.closedChildren}::tail2

We want to 'fold' across the commands and get an updated process status each time.

In [7]:
let processCommand (status: ProcessStatus option) (command: Command): ProcessStatus option =
    match status with
    | None -> None // command processing has already failed
    | Some status ->
        printfn "stack: %A (max %d)" (status.stack |> Seq.map boxSummary) status.maxDepth
        printfn "command: %A" command // TODO: remove debugging
        let stack = status.stack
        match command with
        | Unknown line -> None // TODO
        | Close closeSize ->
            if (stack.Head.state = Closed)
            then None // can't close a box that is already closed
            else
                if (closeSize <> stack.Head.size)
                then None // trying to close the wrong size of box
                else
                    match stack with
                    | [] -> None
                    | head::tail ->
                        if (tail.IsEmpty)
                        then
                            let closedHead = {head with state = Closed}
                            Some {status with stack = [closedHead]} // always leave the last element on the stack
                        else
                            let newStack = closeStackHead stack
                            match newStack.Head.openedChildren with
                            | [] -> Some {status with stack = newStack}
                            | openHead::openTail ->
                                Some {status with stack = openHead::newStack}
        | Open openSize ->
            let stack = status.stack
            match stack with
            | [] ->
                let box = {state = Opened; size = openSize; openedChildren = []; closedChildren = []}
                Some {stack = [box]; maxDepth = 1}
            | head::tail ->
                if (openSize > (sizeRemaining head))
                then None // not enough room in box for new box
                else
                    let box = {state = Opened; size = openSize; openedChildren = []; closedChildren = []}
                    let newHead = {head with openedChildren = box::head.openedChildren}
                    let newStack = box::newHead::tail
                    Some {stack = newStack; maxDepth = List.max [status.maxDepth; newStack.Length]}

## Calculate the Result

In [8]:
let result = commands |> Seq.fold processCommand (Some initProcessStatus)

let finalResult =
    match result with
    | Some status ->
        if ((status.stack.Length = 1) && (status.stack.Head.state = Closed))
        then Some status
        else None
    | None -> None

match finalResult with
| Some status -> printfn "Well done!"
| None -> printfn "You are fired!"

stack: seq [] (max 0)
command: Open 20
stack: seq [('O', 20, 20)] (max 1)
command: Open 10
stack: seq [('O', 10, 10); ('O', 20, 10)] (max 2)
command: Open 5
stack: seq [('O', 5, 5); ('O', 10, 5); ('O', 20, 10)] (max 3)
command: Close 5
stack: seq [('O', 10, 5); ('O', 20, 10)] (max 3)
command: Open 5
stack: seq [('O', 5, 5); ('O', 10, 0); ('O', 20, 10)] (max 3)
command: Open 3
stack: seq [('O', 3, 3); ('O', 5, 2); ('O', 10, 0); ('O', 20, 10)] (max 4)
command: Close 3
stack: seq [('O', 5, 2); ('O', 10, 0); ('O', 20, 10)] (max 4)
command: Open 2
stack: seq [('O', 2, 2); ('O', 5, 0); ('O', 10, 0); ('O', 20, 10)] (max 4)
command: Close 2
stack: seq [('O', 5, 0); ('O', 10, 0); ('O', 20, 10)] (max 4)
command: Close 5
stack: seq [('O', 10, 0); ('O', 20, 10)] (max 4)
command: Close 10
stack: seq [('O', 20, 10)] (max 4)
command: Close 20
Well done!


And if it was a success, print the maximum box depth.

In [9]:
if (finalResult.IsSome)
then
    printfn "Max depth is %d" finalResult.Value.maxDepth

Max depth is 4
